In [1]:
import pyodbc
import pandas as pd

In [8]:
try:
    server = 'MENDONOTE\SQLEXPRESS' # ejemplo
    base = input("Base: ")
    usuario = input("Usuario: ")
    contraseña = input("Contraseña: ")
    database = base
    username = usuario 
    password = contraseña
    coneccion = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    print("Conexión exitosa")
    cursor = coneccion.cursor()
    cursor.execute("SELECT @@version;") 
    row = cursor.fetchone()
    print(row)
    sql = """SELECT
            	MONTH(vc.fecha) AS "MES",
                YEAR(vc.fecha) AS "AÑO",
                s.denominacion AS "SUCURSAL",
                SUM(vd.precio * vd.cantidad) AS "GANANCIAS"
            FROM
            	vencab AS vc INNER JOIN vendet AS vd ON (vc.letra = vd.letra AND vc.factura = vd.factura)
            	INNER JOIN sucursales AS s ON vc.sucursal = s.sucursal
            WHERE
            	vc.anulada=0
            GROUP BY
            	MONTH(vc.fecha) ,
            	YEAR(vc.fecha) ,
            	s.denominacion
            HAVING 
            	SUM(vd.precio * vd.cantidad) > 10
            ORDER BY
            	3, 2, 1"""
                
    cursor.execute(sql)
    rows=cursor.fetchall()
    lista=[]
    for fila in rows:
        #lista.append(fila)
        lista.append((fila[0], fila[1], fila[2].strip(), float(fila[3])))
    
except Exception as ex:
    print(ex)

#print(lista)
df = pd.DataFrame(lista,columns=['Mes','Año','Sucursal','Ganancias'])

Base: Ventas2
Usuario: sa
Contraseña: password
Conexión exitosa
('Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) \n\tSep 24 2019 13:48:23 \n\tCopyright (C) 2019 Microsoft Corporation\n\tExpress Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 19042: )\n', )


In [11]:
df[(df.Sucursal=="9 DE JULIO")&(df.Año==2009)]

,Mes,Año,Sucursal,Ganancias
58,1,2009,9 DE JULIO,134682.3
59,2,2009,9 DE JULIO,24427.0


In [10]:
df

,Mes,Año,Sucursal,Ganancias
0,3,2004,9 DE JULIO,12635.25
1,4,2004,9 DE JULIO,74388.05
2,5,2004,9 DE JULIO,99807.58
3,6,2004,9 DE JULIO,91464.65
4,7,2004,9 DE JULIO,99101.80
...,...,...,...,...
672,10,2008,VILLA RIVERA INDARTE AF,50379.86
673,11,2008,VILLA RIVERA INDARTE AF,69848.00
674,12,2008,VILLA RIVERA INDARTE AF,74140.30
675,1,2009,VILLA RIVERA INDARTE AF,41510.51


In [4]:
df=df[(df.Sucursal.isin(["9 DE JULIO","ITUZAINGO","PATIO OLMOS","VILLA CABRERA"]))&(df.Año>2004)&(df.Año<2009)]
df

,Mes,Año,Sucursal,Ganancias
10,1,2005,9 DE JULIO,63220.90
11,2,2005,9 DE JULIO,86684.00
12,3,2005,9 DE JULIO,83729.02
13,4,2005,9 DE JULIO,110671.50
14,5,2005,9 DE JULIO,103475.77
...,...,...,...,...
508,8,2008,VILLA CABRERA,91642.52
509,9,2008,VILLA CABRERA,82501.51
510,10,2008,VILLA CABRERA,75828.41
511,11,2008,VILLA CABRERA,65309.10


In [5]:
df=pd.get_dummies(df, columns = ['Sucursal'])
df

,Mes,Año,Ganancias,Sucursal_9 DE JULIO,Sucursal_ITUZAINGO,Sucursal_PATIO OLMOS,Sucursal_VILLA CABRERA
10,1,2005,63220.90,1,0,0,0
11,2,2005,86684.00,1,0,0,0
12,3,2005,83729.02,1,0,0,0
13,4,2005,110671.50,1,0,0,0
14,5,2005,103475.77,1,0,0,0
...,...,...,...,...,...,...,...
508,8,2008,91642.52,0,0,0,1
509,9,2008,82501.51,0,0,0,1
510,10,2008,75828.41,0,0,0,1
511,11,2008,65309.10,0,0,0,1


In [6]:
df=df[["Mes","Año","Sucursal_9 DE JULIO","Sucursal_ITUZAINGO", "Sucursal_PATIO OLMOS","Sucursal_VILLA CABRERA","Ganancias"]]
df

,Mes,Año,Sucursal_9 DE JULIO,Sucursal_ITUZAINGO,Sucursal_PATIO OLMOS,Sucursal_VILLA CABRERA,Ganancias
10,1,2005,1,0,0,0,63220.90
11,2,2005,1,0,0,0,86684.00
12,3,2005,1,0,0,0,83729.02
13,4,2005,1,0,0,0,110671.50
14,5,2005,1,0,0,0,103475.77
...,...,...,...,...,...,...,...
508,8,2008,0,0,0,1,91642.52
509,9,2008,0,0,0,1,82501.51
510,10,2008,0,0,0,1,75828.41
511,11,2008,0,0,0,1,65309.10


In [7]:
df.to_csv('baseTP.csv', index=False)